In [20]:
# import matplotlib.pyplot as plt
import numpy as np
def cos_edge_square(twidth,  dt, ramp_fraction=0.25):
    tedge=np.arange(0, 2*twidth*ramp_fraction, dt)
    t=np.arange(0, twidth, dt)
    width=len(t)
    if (ramp_fraction>0 and ramp_fraction<=0.5 and twidth>0):
        f=1.0/(2*ramp_fraction*twidth)
        edges=(np.cos(2*np.pi*f*tedge-np.pi)+1.0)/2.0
        #pyplot.plot(edges)
        #pyplot.show()
        nramp=int(len(edges)/2)
        nflat=width-len(edges)
        env=np.concatenate((edges[:nramp], np.ones(nflat), edges[nramp:]))
    else:
        print('ramp_fraction (ramp_length/twidth) should be 0<ramp_function<=0.5, %s and twidth>0 %s'%(str(ramp_fraction), str(twidth)))
        env=np.ones(width)
    return (t, env)

# def test():
#     t, env = cos_edge_square(128, 1)
#     plt.plot(t, env * 0.9)
#     plt.show()

# test()


In [ ]:
# utils for simulation

from riscq.assembler import *
from pynq import Overlay
overlay = Overlay("./qubic-2-qubits.bit")

from pynq import MMIO
pulse_mem_offset = 0x80000000
PULSE_MEM = MMIO(pulse_mem_offset, 2 ** 24)
inst_offset = 0x81000000
INST_MEM = MMIO(inst_offset, 2 ** 24)
rst_offset = 0x90000000
RST_MEM = MMIO(rst_offset, 2 ** 8)

def reset_dsp():
  RST_MEM.write(0, 1)
  RST_MEM.write(0, 0)

def load_inst(addr, insts):
  bins = assemble(insts)
  cur_addr = addr
  for inst in bins:
    slices = [inst[i:i+32] for i in range(0, 128, 32)]
    for slice in reversed(slices):
      INST_MEM.write(cur_addr, int(slice, 2).to_bytes(4, 'little'))
      cur_addr += 4

def load_pulse(id, addr, env):
  cur_addr = id * (2 ** 20) + addr
  for d in env:
    d_int = int(d * (2 ** 15))
    d_bytes = d_int.to_bytes(2, 'little', signed = True)
    PULSE_MEM.write(cur_addr, d_bytes)
    print(d_int, d_bytes)
    cur_addr += 2

In [ ]:
# prepare data

## prepare instructions
set_time = SETTIME(10)

# the phase between each env point is freq / (2 ** 15) * pi
inv_freq = int((1 << 15) / 32.0)
amp = (1 << 15) - 1
id = 0
set_carrier = CARRIER(inv_freq = inv_freq, phase = 0, id = id)
send_pulse = PULSE(start = 70, addr = 0, duration = 8, phase = 0, amp = amp, id = id)
loop = BEQ(0, 0, 0)

insts = [set_time, set_carrier, send_pulse, loop]

## prepare envelope
t, env = cos_edge_square(128, 1)


In [ ]:
# start test

reset_dsp()

load_inst(0, insts)
load_pulse(0, 0, env * 0.9)

reset_dsp()
